In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import sys

if len(sys.argv) == 2:
    fn = sys.argv[1]
else:
    from pathlib import Path

    paths = []
    for p in (Path("..") / ".benchmarks").rglob("*.json"):
        paths.append((p.stat().st_mtime, p))
    paths.sort()
    fn = paths[-1][1]

with open(fn) as f:
    data = json.load(f)

print(
    f"""\
benchmark results
  {data['datetime']}
  {data['machine_info']['cpu']['brand_raw']}
"""
)

variant = {}
for b in data["benchmarks"]:
    params = b["params"]
    n = params["n"]
    n = int(n)
    name = b["name"]
    name = name[name.find("_") + 1 : name.find("[")]
    extra = [k for (k, v) in params.items() if k not in ("n", "lib", "model") and v]
    if extra:
        name += "_" + "_".join(extra)
    for key in ("lib", "model"):
        if key in params:
            name += f"_{params[key]}"
    t = b["stats"]["min"]
    if name not in variant:
        variant[name] = []
    variant[name].append((n, t))

for k in variant:
    print(k)

names = {
    "Numba vs. SciPy": {
        "nll_scipy.stats": "handwritten NLL scipy.stats",
        "nll_numba_stats": "handwritten NLL numba_stats",
    },
    "Overhead": {
        "minuit_UnbinnedNLL": "iminuit+UnbinnedNLL",
        "minuit_numba": "iminuit+numba",
        "minuit_cfunc": "iminuit+numba.cfunc",
    },
    "RooFit vs. iminuit+numba": {
        "RooFit": "RooFit",
        "RooFit_BatchMode": "RooFit with BatchMode",
        "RooFit_NumCPU": "RooFit with NumCPU",
        "minuit": "iminuit+numba",
        "minuit_parallel_fastmath": "iminuit+numba with parallel, fastmath",
        "minuit_parallel_fastmath_log": "iminuit+numba logpdf with parallel, fastmath",
    },
}

for title, subnames in names.items():
    plt.figure(constrained_layout=True)
    plt.title(title)
    for name in subnames:
        if name not in variant:
            continue
        d = variant[name]
        n, t = np.transpose(d)
        ls = "-"
        if (
            "parallel" in name
            and "fastmath" in name
            or ("NumCPU" in name and "BatchMode" in name)
        ):
            ls = "-."
        elif "parallel" in name or "NumCPU" in name:
            ls = "--"
        elif "fastmath" in name or "BatchMode" in name:
            ls = ":"
        plt.plot(n, t, ls=ls, label=subnames[name])

    plt.loglog()
    plt.legend(
        frameon=False,
        fontsize="medium" if len(subnames) < 4 else "small",
        ncol=1 if len(subnames) < 3 else 2,
    )
    # plt.title("Fit of normal distribution with 2 parameters")
    plt.xlabel("number of data points")
    plt.ylabel("runtime / sec")
    fname = title.replace(" ", "_").replace(".", "_").replace("__", "_").lower()
    plt.savefig(f"{fname}.svg")
